# 🎯 Multi-Modal Violence Detection
## CNN + BiLSTM with Pose Detection and Emotion Recognition

**Expected Performance:** 92-97% accuracy on RWF-2000 dataset

**⚡ WORKFLOW:**
1. **Add Dataset**: Add `vulamnguyen/rwf2000` dataset to notebook
2. **Enable GPU**: Settings → Accelerator → GPU T4 x2
3. **Run All**: Cell → Run All
4. **Wait 4-6 hours**: Fully automated!
5. **Download Model**: Auto-saved to `/kaggle/working/`

**Total Time:** ~4-6 hours with GPU

---

## 📋 Section 1: Setup GPU and Install Packages

In [ ]:
# Check GPU availability
import tensorflow as tf
print("="*70)
print("🖥️  SYSTEM INFORMATION")
print("="*70)
print(f"TensorFlow Version: {tf.__version__}")
print(f"GPU Available: {len(tf.config.list_physical_devices('GPU')) > 0}")
print(f"GPU Devices: {tf.config.list_physical_devices('GPU')}")
print("="*70)

In [ ]:
# Install required packages
print("\n📦 Installing required packages...\n")

!pip install -q opencv-python-headless
!pip install -q mediapipe
!pip install -q deepface
!pip install -q scikit-learn matplotlib seaborn tqdm

print("\n✅ All packages installed successfully!")

## 📂 Section 2: Locate Dataset (Kaggle Auto-Mounted)

In [ ]:
import os

print("="*70)
print("📂 LOCATING RWF-2000 DATASET")
print("="*70)

# Kaggle datasets are in /kaggle/input/
possible_paths = [
    '/kaggle/input/rwf2000/RWF-2000',
    '/kaggle/input/rwf2000',
    '/kaggle/input/rwf-2000/RWF-2000',
    '/kaggle/input/rwf-2000',
    '/kaggle/input/RWF-2000',
]

DATASET_PATH = None

for path in possible_paths:
    if os.path.exists(path):
        check_train = os.path.join(path, 'train')
        check_val = os.path.join(path, 'val')
        if os.path.exists(check_train) and os.path.exists(check_val):
            DATASET_PATH = path
            break

# If not found, search entire /kaggle/input/
if DATASET_PATH is None:
    print("🔍 Searching /kaggle/input/ directory...")
    for root, dirs, files in os.walk('/kaggle/input'):
        if 'train' in dirs and 'val' in dirs:
            train_path = os.path.join(root, 'train')
            if os.path.exists(os.path.join(train_path, 'Fight')):
                DATASET_PATH = root
                break

if DATASET_PATH is None:
    print("\n❌ ERROR: RWF-2000 dataset not found!")
    print("\n💡 Please add the dataset:")
    print("   1. Click 'Add Data' button (right sidebar)")
    print("   2. Search for 'rwf2000' or 'vulamnguyen/rwf2000'")
    print("   3. Click 'Add' button")
    print("   4. Re-run this cell")
    print("\n📁 Available datasets:")
    !ls -la /kaggle/input/
    raise FileNotFoundError("RWF-2000 dataset not found in /kaggle/input/")

print(f"\n✅ Dataset found at: {DATASET_PATH}")

# Setup paths
CACHE_PATH = '/kaggle/working/violence_detection_cache'
MODEL_SAVE_PATH = '/kaggle/working/violence_detection_models'

os.makedirs(CACHE_PATH, exist_ok=True)
os.makedirs(MODEL_SAVE_PATH, exist_ok=True)

print(f"✅ Cache directory: {CACHE_PATH}")
print(f"✅ Models directory: {MODEL_SAVE_PATH}")
print("="*70)

In [ ]:
# Verify dataset structure
print("\n" + "="*70)
print("📊 VERIFYING DATASET STRUCTURE")
print("="*70)

train_fight = os.path.join(DATASET_PATH, 'train', 'Fight')
train_nonfight = os.path.join(DATASET_PATH, 'train', 'NonFight')
val_fight = os.path.join(DATASET_PATH, 'val', 'Fight')
val_nonfight = os.path.join(DATASET_PATH, 'val', 'NonFight')

paths = {
    'Train/Fight': train_fight,
    'Train/NonFight': train_nonfight,
    'Val/Fight': val_fight,
    'Val/NonFight': val_nonfight
}

total_videos = 0
for name, path in paths.items():
    if os.path.exists(path):
        videos = [f for f in os.listdir(path) if f.endswith(('.avi', '.mp4'))]
        count = len(videos)
        total_videos += count
        print(f"✅ {name}: {count} videos")
    else:
        print(f"❌ {name}: NOT FOUND!")

print("="*70)
print(f"Total videos: {total_videos}")
if total_videos == 2000:
    print("✅ Dataset structure is CORRECT!")
else:
    print(f"⚠️ Expected 2000 videos, but found {total_videos}")
print("="*70)

## 📚 Section 3: Import Libraries

In [ ]:
# Import all required libraries
import numpy as np
import cv2
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.notebook import tqdm
from datetime import datetime
import json

# Deep Learning
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, Model
from tensorflow.keras.applications import MobileNetV2

# Pose and Emotion Detection
import mediapipe as mp
from deepface import DeepFace

# Metrics
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc

print("✅ All libraries imported successfully!")

## ⚙️ Section 4: Configuration

In [ ]:
# Configuration class
class Config:
    # Paths
    DATASET_DIR = DATASET_PATH
    CACHE_DIR = CACHE_PATH
    MODEL_SAVE_DIR = MODEL_SAVE_PATH
    
    # Video processing
    SEQUENCE_LENGTH = 20
    IMG_HEIGHT = 224
    IMG_WIDTH = 224
    IMG_CHANNELS = 3
    
    # Features
    POSE_DIM = 120
    EMOTION_DIM = 8
    
    # Training
    BATCH_SIZE = 32
    EPOCHS = 30
    LEARNING_RATE = 1e-4
    
    # Model
    LSTM_UNITS = 256
    FUSION_TYPE = 'adaptive'
    
    # Callbacks
    PATIENCE_EARLY_STOP = 8
    PATIENCE_REDUCE_LR = 4

config = Config()

print("="*70)
print("📊 CONFIGURATION")
print("="*70)
print(f"Sequence Length: {config.SEQUENCE_LENGTH} frames")
print(f"Image Size: {config.IMG_HEIGHT}x{config.IMG_WIDTH}")
print(f"Batch Size: {config.BATCH_SIZE}")
print(f"Epochs: {config.EPOCHS}")
print(f"Learning Rate: {config.LEARNING_RATE}")
print("="*70)

## 🎬 Section 5: Pose and Emotion Detection Implementation

In [ ]:
# Pose and Emotion Preprocessor Class
class PoseEmotionPreprocessor:
    def __init__(self):
        self.mp_pose = mp.solutions.pose
        self.pose_detector = self.mp_pose.Pose(
            static_image_mode=False,
            model_complexity=1,
            min_detection_confidence=0.5,
            min_tracking_confidence=0.5
        )
    
    def _calculate_angle(self, a, b, c):
        a, b, c = np.array(a), np.array(b), np.array(c)
        radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
        angle = np.abs(radians*180.0/np.pi)
        if angle > 180.0:
            angle = 360 - angle
        return angle
    
    def extract_pose_features_advanced(self, frame):
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = self.pose_detector.process(frame_rgb)
        
        if results.pose_landmarks:
            landmarks = results.pose_landmarks.landmark
            
            # Basic keypoints
            keypoints = []
            for lm in landmarks:
                keypoints.extend([lm.x, lm.y, lm.visibility])
            
            # Joint angles
            left_elbow = self._calculate_angle(
                [landmarks[11].x, landmarks[11].y],
                [landmarks[13].x, landmarks[13].y],
                [landmarks[15].x, landmarks[15].y]
            )
            right_elbow = self._calculate_angle(
                [landmarks[12].x, landmarks[12].y],
                [landmarks[14].x, landmarks[14].y],
                [landmarks[16].x, landmarks[16].y]
            )
            left_knee = self._calculate_angle(
                [landmarks[23].x, landmarks[23].y],
                [landmarks[25].x, landmarks[25].y],
                [landmarks[27].x, landmarks[27].y]
            )
            right_knee = self._calculate_angle(
                [landmarks[24].x, landmarks[24].y],
                [landmarks[26].x, landmarks[26].y],
                [landmarks[28].x, landmarks[28].y]
            )
            left_shoulder = self._calculate_angle(
                [landmarks[13].x, landmarks[13].y],
                [landmarks[11].x, landmarks[11].y],
                [landmarks[23].x, landmarks[23].y]
            )
            right_shoulder = self._calculate_angle(
                [landmarks[14].x, landmarks[14].y],
                [landmarks[12].x, landmarks[12].y],
                [landmarks[24].x, landmarks[24].y]
            )
            
            # Body metrics
            hand_distance = np.sqrt(
                (landmarks[15].x - landmarks[16].x)**2 +
                (landmarks[15].y - landmarks[16].y)**2
            )
            foot_elevation = abs(landmarks[27].y - landmarks[28].y)
            torso_bend_left = abs(landmarks[11].y - landmarks[23].y)
            torso_bend_right = abs(landmarks[12].y - landmarks[24].y)
            center_x = np.mean([landmarks[11].x, landmarks[12].x, landmarks[23].x, landmarks[24].x])
            center_y = np.mean([landmarks[11].y, landmarks[12].y, landmarks[23].y, landmarks[24].y])
            head_offset_x = landmarks[0].x - center_x
            head_offset_y = landmarks[0].y - center_y
            
            advanced_features = keypoints + [
                left_elbow, right_elbow, left_knee, right_knee,
                left_shoulder, right_shoulder, hand_distance, foot_elevation,
                torso_bend_left, torso_bend_right, center_x, center_y,
                head_offset_x, head_offset_y,
                left_elbow/180, right_elbow/180, left_knee/180, right_knee/180,
                left_shoulder/180, right_shoulder/180, hand_distance * 10
            ]
            
            return np.array(advanced_features[:120])
        else:
            return np.zeros(120)
    
    def extract_emotion_features(self, frame):
        try:
            analysis = DeepFace.analyze(
                frame,
                actions=['emotion'],
                enforce_detection=False,
                silent=True
            )
            
            if isinstance(analysis, list):
                analysis = analysis[0]
            
            emotions = analysis['emotion']
            emotion_vector = [
                emotions.get('angry', 0.0) / 100.0,
                emotions.get('disgust', 0.0) / 100.0,
                emotions.get('fear', 0.0) / 100.0,
                emotions.get('happy', 0.0) / 100.0,
                emotions.get('sad', 0.0) / 100.0,
                emotions.get('surprise', 0.0) / 100.0,
                emotions.get('neutral', 0.0) / 100.0
            ]
            variance = np.var(emotion_vector)
            return np.array(emotion_vector + [variance])
        except:
            return np.zeros(8)
    
    def extract_enhanced_features(self, video_path, num_frames=20, target_size=(224, 224)):
        cap = cv2.VideoCapture(video_path)
        total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        
        if total_frames == 0:
            cap.release()
            raise ValueError(f"No frames in video: {video_path}")
        
        if total_frames >= num_frames:
            indices = np.linspace(0, total_frames - 1, num_frames, dtype=int)
        else:
            indices = list(range(total_frames)) + [total_frames - 1] * (num_frames - total_frames)
        
        frames = []
        poses = []
        emotions = []
        
        for idx in indices:
            cap.set(cv2.CAP_PROP_POS_FRAMES, idx)
            ret, frame = cap.read()
            
            if ret:
                frame_resized = cv2.resize(frame, target_size)
                frames.append(frame_resized)
                poses.append(self.extract_pose_features_advanced(frame))
                emotions.append(self.extract_emotion_features(frame))
            else:
                if len(frames) > 0:
                    frames.append(frames[-1])
                    poses.append(poses[-1])
                    emotions.append(emotions[-1])
                else:
                    frames.append(np.zeros((*target_size, 3), dtype=np.uint8))
                    poses.append(np.zeros(120))
                    emotions.append(np.zeros(8))
        
        cap.release()
        
        return {
            'frames': np.array(frames),
            'pose': np.array(poses),
            'emotion': np.array(emotions)
        }

print("✅ PoseEmotionPreprocessor class defined!")

## 📂 Section 6: Load Dataset Paths

In [ ]:
def load_dataset_paths(dataset_dir):
    train_fight_dir = os.path.join(dataset_dir, 'train', 'Fight')
    train_nonfight_dir = os.path.join(dataset_dir, 'train', 'NonFight')
    val_fight_dir = os.path.join(dataset_dir, 'val', 'Fight')
    val_nonfight_dir = os.path.join(dataset_dir, 'val', 'NonFight')
    
    train_fight = sorted([os.path.join(train_fight_dir, f) for f in os.listdir(train_fight_dir) if f.endswith(('.avi', '.mp4'))])
    train_nonfight = sorted([os.path.join(train_nonfight_dir, f) for f in os.listdir(train_nonfight_dir) if f.endswith(('.avi', '.mp4'))])
    val_fight = sorted([os.path.join(val_fight_dir, f) for f in os.listdir(val_fight_dir) if f.endswith(('.avi', '.mp4'))])
    val_nonfight = sorted([os.path.join(val_nonfight_dir, f) for f in os.listdir(val_nonfight_dir) if f.endswith(('.avi', '.mp4'))])
    
    train_paths = train_fight + train_nonfight
    train_labels = [1] * len(train_fight) + [0] * len(train_nonfight)
    
    val_paths = val_fight + val_nonfight
    val_labels = [1] * len(val_fight) + [0] * len(val_nonfight)
    
    print("="*70)
    print("DATASET LOADED")
    print("="*70)
    print(f"Training: {len(train_paths)} videos ({len(train_fight)} Fight, {len(train_nonfight)} Non-Fight)")
    print(f"Validation: {len(val_paths)} videos ({len(val_fight)} Fight, {len(val_nonfight)} Non-Fight)")
    print("="*70)
    
    return train_paths, train_labels, val_paths, val_labels

train_paths, train_labels, val_paths, val_labels = load_dataset_paths(config.DATASET_DIR)

## ⚡ Section 7: Preprocess and Cache Features

In [ ]:
def preprocess_and_cache_dataset(video_paths, labels, cache_prefix, preprocessor):
    cache_file = os.path.join(config.CACHE_DIR, f'{cache_prefix}_features.npz')
    
    if os.path.exists(cache_file):
        print(f"✅ Loading cached features from: {cache_file}")
        data = np.load(cache_file)
        return data['frames'], data['pose'], data['emotion'], data['labels']
    
    print(f"\n🔄 Processing {len(video_paths)} videos (2-3 hours)...")
    print(f"   Progress will be saved to: {cache_file}")
    
    all_frames = []
    all_poses = []
    all_emotions = []
    all_labels = []
    failed = 0
    
    for i, (video_path, label) in enumerate(tqdm(zip(video_paths, labels), total=len(video_paths), desc="Extracting features")):
        try:
            features = preprocessor.extract_enhanced_features(
                video_path,
                num_frames=config.SEQUENCE_LENGTH,
                target_size=(config.IMG_HEIGHT, config.IMG_WIDTH)
            )
            
            if features is not None:
                all_frames.append(features['frames'].astype(np.float32) / 255.0)
                all_poses.append(features['pose'].astype(np.float32))
                all_emotions.append(features['emotion'].astype(np.float32))
                all_labels.append(label)
        except Exception as e:
            print(f"\n⚠️ Error processing {os.path.basename(video_path)}: {str(e)}")
            failed += 1
    
    print(f"\n✅ Processed {len(all_frames)} videos successfully")
    if failed > 0:
        print(f"⚠️ Failed: {failed} videos")
    
    all_frames = np.array(all_frames, dtype=np.float32)
    all_poses = np.array(all_poses, dtype=np.float32)
    all_emotions = np.array(all_emotions, dtype=np.float32)
    all_labels = np.array(all_labels, dtype=np.float32)
    
    print(f"\n💾 Saving features to cache...")
    np.savez_compressed(
        cache_file,
        frames=all_frames,
        pose=all_poses,
        emotion=all_emotions,
        labels=all_labels
    )
    
    print(f"✅ Features cached!")
    print(f"   Frames: {all_frames.shape}")
    print(f"   Pose: {all_poses.shape}")
    print(f"   Emotion: {all_emotions.shape}")
    
    return all_frames, all_poses, all_emotions, all_labels

print("✅ Preprocessing function defined!")

In [ ]:
# Initialize preprocessor and process datasets
preprocessor = PoseEmotionPreprocessor()

print("\n" + "="*70)
print("PREPROCESSING TRAINING DATA")
print("="*70)
train_frames, train_pose, train_emotion, train_labels_array = preprocess_and_cache_dataset(
    train_paths, train_labels, 'train', preprocessor
)

print("\n" + "="*70)
print("PREPROCESSING VALIDATION DATA")
print("="*70)
val_frames, val_pose, val_emotion, val_labels_array = preprocess_and_cache_dataset(
    val_paths, val_labels, 'val', preprocessor
)

print("\n" + "="*70)
print("✅ PREPROCESSING COMPLETE!")
print("="*70)
print(f"Training samples: {len(train_frames)}")
print(f"Validation samples: {len(val_frames)}")
print("="*70)

## 🏗️ Section 8: Build Multi-Modal Model

In [ ]:
# Custom Attention Layer
class AttentionLayer(layers.Layer):
    def __init__(self, units=128, **kwargs):
        super(AttentionLayer, self).__init__(**kwargs)
        self.units = units
    
    def build(self, input_shape):
        self.W = self.add_weight(name='attention_weight', shape=(input_shape[-1], self.units),
                                initializer='glorot_uniform', trainable=True)
        self.b = self.add_weight(name='attention_bias', shape=(self.units,),
                                initializer='zeros', trainable=True)
        self.u = self.add_weight(name='attention_context', shape=(self.units,),
                                initializer='glorot_uniform', trainable=True)
        super(AttentionLayer, self).build(input_shape)
    
    def call(self, x):
        uit = tf.tanh(tf.tensordot(x, self.W, axes=1) + self.b)
        ait = tf.tensordot(uit, self.u, axes=1)
        attention_weights = tf.nn.softmax(ait, axis=1)
        attention_weights = tf.expand_dims(attention_weights, -1)
        weighted_input = x * attention_weights
        return tf.reduce_sum(weighted_input, axis=1)
    
    def get_config(self):
        config_dict = super(AttentionLayer, self).get_config()
        config_dict.update({'units': self.units})
        return config_dict

print("✅ AttentionLayer defined!")

In [ ]:
def build_multimodal_model():
    # INPUT LAYERS
    frame_input = layers.Input(shape=(config.SEQUENCE_LENGTH, config.IMG_HEIGHT, config.IMG_WIDTH, 3), name='frames')
    pose_input = layers.Input(shape=(config.SEQUENCE_LENGTH, config.POSE_DIM), name='pose')
    emotion_input = layers.Input(shape=(config.SEQUENCE_LENGTH, config.EMOTION_DIM), name='emotion')
    
    # RGB BRANCH
    mobilenet = MobileNetV2(input_shape=(config.IMG_HEIGHT, config.IMG_WIDTH, 3),
                            include_top=False, weights='imagenet', pooling='avg')
    mobilenet.trainable = False
    
    x_rgb = layers.TimeDistributed(mobilenet, name='mobilenet')(frame_input)
    x_rgb = layers.Dropout(0.3)(x_rgb)
    x_rgb = layers.Bidirectional(layers.LSTM(config.LSTM_UNITS, return_sequences=True, 
                                              dropout=0.3, recurrent_dropout=0.2), name='rgb_bilstm')(x_rgb)
    x_rgb = layers.Dropout(0.4)(x_rgb)
    rgb_output = AttentionLayer(128, name='rgb_attention')(x_rgb)
    rgb_output = layers.Dense(256, activation='relu')(rgb_output)
    rgb_output = layers.BatchNormalization()(rgb_output)
    rgb_output = layers.Dropout(0.5)(rgb_output)
    
    # POSE BRANCH
    x_pose = layers.BatchNormalization()(pose_input)
    x_pose = layers.TimeDistributed(layers.Dense(128, activation='relu'))(x_pose)
    x_pose = layers.Dropout(0.3)(x_pose)
    x_pose = layers.Bidirectional(layers.LSTM(128, return_sequences=True, 
                                               dropout=0.3, recurrent_dropout=0.2), name='pose_bilstm')(x_pose)
    x_pose = layers.Dropout(0.4)(x_pose)
    pose_output = AttentionLayer(64, name='pose_attention')(x_pose)
    pose_output = layers.Dense(128, activation='relu')(pose_output)
    pose_output = layers.BatchNormalization()(pose_output)
    pose_output = layers.Dropout(0.5)(pose_output)
    
    # EMOTION BRANCH
    x_emotion = layers.BatchNormalization()(emotion_input)
    x_emotion = layers.TimeDistributed(layers.Dense(64, activation='relu'))(x_emotion)
    x_emotion = layers.Dropout(0.3)(x_emotion)
    x_emotion = layers.Bidirectional(layers.LSTM(64, return_sequences=True,
                                                  dropout=0.3, recurrent_dropout=0.2), name='emotion_bilstm')(x_emotion)
    x_emotion = layers.Dropout(0.4)(x_emotion)
    emotion_output = layers.GlobalAveragePooling1D()(x_emotion)
    emotion_output = layers.Dense(64, activation='relu')(emotion_output)
    emotion_output = layers.BatchNormalization()(emotion_output)
    emotion_output = layers.Dropout(0.5)(emotion_output)
    
    # ADAPTIVE FUSION
    rgb_proj = layers.Dense(256, activation='relu', name='rgb_proj')(rgb_output)
    pose_proj = layers.Dense(256, activation='relu', name='pose_proj')(pose_output)
    emotion_proj = layers.Dense(256, activation='relu', name='emotion_proj')(emotion_output)
    
    stacked = tf.stack([rgb_proj, pose_proj, emotion_proj], axis=1, name='feature_stack')
    fusion_weights = layers.Dense(3, activation='softmax', name='fusion_weights')(layers.Flatten()(stacked))
    fusion_weights = layers.Reshape((3, 1))(fusion_weights)
    weighted_features = layers.Multiply()([stacked, fusion_weights])
    fused = layers.Lambda(lambda x: tf.reduce_sum(x, axis=1), name='fusion_sum')(weighted_features)
    
    # CLASSIFICATION HEAD
    x = layers.Dense(512, activation='relu')(fused)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(0.5)(x)
    x = layers.Dense(256, activation='relu')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(0.5)(x)
    output = layers.Dense(1, activation='sigmoid', name='output')(x)
    
    model = Model(inputs=[frame_input, pose_input, emotion_input], outputs=output,
                  name='multimodal_violence_detector')
    
    return model

model = build_multimodal_model()

model.compile(
    optimizer=keras.optimizers.Adam(config.LEARNING_RATE),
    loss='binary_crossentropy',
    metrics=['accuracy', keras.metrics.Precision(name='precision'),
             keras.metrics.Recall(name='recall'), keras.metrics.AUC(name='auc')]
)

print("\n" + "="*70)
print("MODEL BUILT & COMPILED")
print("="*70)
model.summary()
print("="*70)

## 🏋️ Section 9: Train Model

In [ ]:
# Setup callbacks
checkpoint = keras.callbacks.ModelCheckpoint(
    filepath=os.path.join(config.MODEL_SAVE_DIR, 'best_multimodal_model.h5'),
    monitor='val_accuracy',
    save_best_only=True,
    mode='max',
    verbose=1
)

early_stop = keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=config.PATIENCE_EARLY_STOP,
    restore_best_weights=True,
    verbose=1
)

reduce_lr = keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,
    patience=config.PATIENCE_REDUCE_LR,
    min_lr=1e-7,
    verbose=1
)

# Class weights
unique, counts = np.unique(train_labels_array, return_counts=True)
class_weights = {int(cls): len(train_labels_array) / (len(unique) * count) for cls, count in zip(unique, counts)}

print("="*70)
print("TRAINING CONFIGURATION")
print("="*70)
print(f"Class weights: {class_weights}")
print("="*70)

In [ ]:
print("\n" + "="*70)
print("🚀 STARTING TRAINING")
print("="*70)
print(f"Training samples: {len(train_frames)}")
print(f"Validation samples: {len(val_frames)}")
print(f"Batch size: {config.BATCH_SIZE}")
print(f"Epochs: {config.EPOCHS}")
print("="*70 + "\n")

history = model.fit(
    x=[train_frames, train_pose, train_emotion],
    y=train_labels_array,
    validation_data=([val_frames, val_pose, val_emotion], val_labels_array),
    batch_size=config.BATCH_SIZE,
    epochs=config.EPOCHS,
    callbacks=[checkpoint, early_stop, reduce_lr],
    class_weight=class_weights,
    verbose=1
)

# Save final model
final_model_path = os.path.join(config.MODEL_SAVE_DIR, 'final_multimodal_model.h5')
model.save(final_model_path)

# Save history
history_path = os.path.join(config.MODEL_SAVE_DIR, 'training_history.json')
with open(history_path, 'w') as f:
    history_dict = {key: [float(v) for v in values] for key, values in history.history.items()}
    json.dump(history_dict, f, indent=2)

print("\n" + "="*70)
print("✅ TRAINING COMPLETED!")
print("="*70)
print(f"Best Validation Accuracy: {max(history.history['val_accuracy']):.4f}")
print(f"Best Validation AUC: {max(history.history['val_auc']):.4f}")
print("="*70)

## 📊 Section 10: Evaluate Model

In [ ]:
# Plot training history
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

axes[0, 0].plot(history.history['accuracy'], label='Train')
axes[0, 0].plot(history.history['val_accuracy'], label='Val')
axes[0, 0].set_title('Accuracy', fontsize=14, fontweight='bold')
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)

axes[0, 1].plot(history.history['loss'], label='Train')
axes[0, 1].plot(history.history['val_loss'], label='Val')
axes[0, 1].set_title('Loss', fontsize=14, fontweight='bold')
axes[0, 1].legend()
axes[0, 1].grid(True, alpha=0.3)

axes[1, 0].plot(history.history['precision'], label='Train')
axes[1, 0].plot(history.history['val_precision'], label='Val')
axes[1, 0].set_title('Precision', fontsize=14, fontweight='bold')
axes[1, 0].legend()
axes[1, 0].grid(True, alpha=0.3)

axes[1, 1].plot(history.history['recall'], label='Train')
axes[1, 1].plot(history.history['val_recall'], label='Val')
axes[1, 1].set_title('Recall', fontsize=14, fontweight='bold')
axes[1, 1].legend()
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(os.path.join(config.MODEL_SAVE_DIR, 'training_history.png'), dpi=150)
plt.show()

In [ ]:
# Generate predictions
y_pred_proba = model.predict([val_frames, val_pose, val_emotion], batch_size=config.BATCH_SIZE)
y_pred = (y_pred_proba > 0.5).astype(int).flatten()
y_true = val_labels_array.astype(int)

print("\n" + "="*70)
print("CLASSIFICATION REPORT")
print("="*70)
print(classification_report(y_true, y_pred, target_names=['Non-Fight', 'Fight'], digits=4))
print("="*70)

In [ ]:
# Confusion Matrix and ROC
fig, axes = plt.subplots(1, 2, figsize=(15, 6))

cm = confusion_matrix(y_true, y_pred)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=axes[0],
            xticklabels=['Non-Fight', 'Fight'], yticklabels=['Non-Fight', 'Fight'])
axes[0].set_title('Confusion Matrix', fontsize=14, fontweight='bold')
axes[0].set_ylabel('True')
axes[0].set_xlabel('Predicted')

fpr, tpr, _ = roc_curve(y_true, y_pred_proba)
roc_auc = auc(fpr, tpr)
axes[1].plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC (AUC = {roc_auc:.4f})')
axes[1].plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
axes[1].set_title('ROC Curve', fontsize=14, fontweight='bold')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(os.path.join(config.MODEL_SAVE_DIR, 'evaluation.png'), dpi=150)
plt.show()

print(f"\n✅ ROC AUC: {roc_auc:.4f}")

## 🎉 Complete! Model saved to `/kaggle/working/violence_detection_models/`

### Files Saved:
- `best_multimodal_model.h5` - Best model (use this!)
- `final_multimodal_model.h5` - Final epoch model
- `training_history.json` - Training metrics
- `training_history.png` - Training plots
- `evaluation.png` - Confusion matrix & ROC

**Download from the Output tab on the right! →**